In [17]:
import requests
import time
from IPython.display import clear_output
import pandas as pd

columns = [
    'company_ticker',
    'company_name',
    'has_trades',
    'insider_names',
    'insider_titles_unique',
    'insiders',
    'last_filing_date',
    'first_trade_date',
    'last_trade_date',
    'state_of_incorporation',
    'industry_group_name',
    'industry_name',
    'sector_name',
    'subsector_name',
    'previous_shares',
    'remaining_shares',
    'trade_dates',
    'trade_status',
    'trade_types',
    'traded_percentage',
    'traded_share_price',
    'traded_shares',
    'traded_value'
]
all_trades = []
errors = []
headers = {'Accept': 'application/json'}

def print_progress(page, page_count, tries, errors):
    clear_output(wait=False)
    print(f"Current Page : {str(page + 1):>{len(str(page_count + 1))}}\u002F{str(page_count):>{len(str(page_count + 1))}}", end="", flush=True)
    print(f"  [ {((page/(page_count + 1)) * 100): 0.2f}%  ]", flush=True)
    print("{ " + (tries * "O") + ((10 - tries) * ".") + " }" + f"   Errors: {errors}", flush=True)

def scrape(query_limit):
    init_payload = {"insider_titles":[],"trade_types":["p","s"],"filing_date_preset":"custom","trade_date_preset":"custom","group_by":"filing","sort_by":"last_filing_date","page":1,"page_limit":query_limit}
    page_count = requests.post('https://www.benzinga.com/sec/insider-trades/api/insider-trades?', json=init_payload, headers=headers).json()['filings']['total_pages']

    for page in range(1):
        payload = {"insider_titles":[],"trade_types":["p","s"],"filing_date_preset":"custom","trade_date_preset":"custom","group_by":"filing","sort_by":"last_filing_date","page":page,"page_limit":query_limit}

        tries = 0
        looping = True
        while looping and tries < 10:
            try:
                print_progress(page, page_count, tries, errors)
                req = requests.post('https://www.benzinga.com/sec/insider-trades/api/insider-trades?', json=payload, headers=headers)
                res = req.json()['filings']
                looping = False
            except:
                tries += 1
        
        if tries == 10:
            errors.append(page)
            continue

        trade_count = len(res['filings'])

        for t in range(50):
            trade = {}
            for c in range(len(columns)):
                if columns[c] != "insiders":
                    try:
                        if res['filings'][t][columns[c]]:
                            trade[columns[c]] = res['filings'][t][columns[c]]
                        else:
                            trade[columns[c]] = None
                    except:
                        trade[columns[c]] = None
                else:
                    try:
                        trade['is_director'] = res['filings'][t]['insiders'][0]['is_director']
                        trade['is_officer'] = res['filings'][t]['insiders'][0]['is_officer']
                        trade['is_ten_percent_owner'] = res['filings'][t]['insiders'][0]['is_ten_percent_owner']
                    except:
                        trade['is_director'] = False
                        trade['is_officer'] = False
                        trade['is_ten_percent_owner'] = False
            all_trades.append(trade)

scrape(500)

trade_data = pd.DataFrame(all_trades)
pd.DataFrame(trade_data)

Current Page :   1/333  [  0.00%  ]
{ .......... }   Errors: []


,company_ticker,company_name,has_trades,insider_names,insider_titles_unique,is_director,is_officer,is_ten_percent_owner,last_filing_date,first_trade_date,...,subsector_name,previous_shares,remaining_shares,trade_dates,trade_status,trade_types,traded_percentage,traded_share_price,traded_shares,traded_value
0,esmt,"EngageSmart, Inc.",True,Robert P. Bennett,CHIEF EXECUTIVE OFFICER,False,False,False,2023-07-14T20:47:13.000Z,2023-07-05T00:00:00.000Z,...,Business Services,8.944910e+05,8.325800e+05,"[2023-07-12T00:00:00.000Z, 2023-07-05T00:00:00...",SELL,"[m, s]",-6.921366,12.100000,-61911.0,-1.431275e+06
1,crwd,"CrowdStrike Holdings, Inc.",True,George Kurtz,PRESIDENT AND CEO,False,False,False,2023-07-14T20:45:04.000Z,2023-07-13T00:00:00.000Z,...,Business Services,1.119971e+06,1.119971e+06,"[2023-07-13T00:00:00.000Z, 2023-07-13T00:00:00...",SELL-OPTIONS,"[c, s]",NaN,151.038553,NaN,-2.688486e+07
2,cacc,CREDIT ACCEPTANCE CORP,True,Donald A Foss Irrevocable Trust Dated August,None,False,False,False,2023-07-14T20:36:46.000Z,2023-07-12T00:00:00.000Z,...,Non-depository Credit Institutions,1.436327e+06,1.420312e+06,"[2023-07-14T00:00:00.000Z, 2023-07-14T00:00:00...",SELL,[s],-1.114997,545.226613,-16015.0,-8.731804e+06
3,aur,"Aurora Innovation, Inc.",True,Sterling Anderson,DIRECTOR,True,False,False,2023-07-14T20:10:26.000Z,2023-07-13T00:00:00.000Z,...,Business Services,9.289900e+04,9.289900e+04,"[2023-07-14T00:00:00.000Z, 2023-07-14T00:00:00...",SELL-OPTIONS,"[c, s]",NaN,3.324482,NaN,-1.248965e+06
4,dkng,DraftKings Inc.,True,Stanton Dodge,CHIEF LEGAL OFFICER,False,True,False,2023-07-14T20:01:27.000Z,2023-07-12T00:00:00.000Z,...,Amusement And Recreation Services,6.290230e+05,6.290230e+05,"[2023-07-12T00:00:00.000Z, 2023-07-12T00:00:00...",SELL-OPTIONS,"[m, s]",NaN,16.535000,NaN,-5.735804e+06
5,dkng,DraftKings Inc.,True,Erik Bradbury,CHIEF ACCOUNTING OFFICER,False,True,False,2023-07-14T20:01:13.000Z,2023-07-12T00:00:00.000Z,...,Amusement And Recreation Services,2.807000e+04,2.507000e+04,[2023-07-12T00:00:00.000Z],SELL,[s],-10.687567,30.120000,-3000.0,-9.036000e+04
6,ped,PEDEVCO CORP,True,John J Scelfo,None,False,False,False,2023-07-14T19:41:27.000Z,2023-07-13T00:00:00.000Z,...,Oil And Gas Extraction,2.645000e+05,2.245000e+05,[2023-07-13T00:00:00.000Z],SELL,[s],-15.122873,0.919000,-40000.0,-3.676000e+04
7,ped,PEDEVCO CORP,True,Ivar Siem,None,False,False,False,2023-07-14T19:40:37.000Z,2023-07-12T00:00:00.000Z,...,Oil And Gas Extraction,3.821000e+05,3.521000e+05,[2023-07-12T00:00:00.000Z],SELL,[s],-7.851348,0.910000,-30000.0,-2.730000e+04
8,giii,G III APPAREL GROUP LTD /DE/,True,Laura Pomerantz,DIRECTOR,True,False,False,2023-07-14T19:34:43.000Z,2023-07-12T00:00:00.000Z,...,General Merchandise Stores,4.197400e+04,3.107000e+04,[2023-07-12T00:00:00.000Z],SELL,[s],-25.977986,20.420000,-10904.0,-2.226597e+05
9,lark,LANDMARK BANCORP INC,True,Richard Ball,DIRECTOR,True,False,False,2023-07-14T19:34:34.000Z,2023-07-12T00:00:00.000Z,...,Depository Institutions,1.765100e+05,1.780760e+05,"[2023-07-14T00:00:00.000Z, 2023-07-13T00:00:00...",BUY,[p],0.887202,21.549656,1566.0,3.374676e+04
